In [7]:
import numpy as np
import pandas as pd

In [149]:
sell_orderbook = np.array([[100, 5, 1, 0], 
                            [99, 3, 2, 1],
                           [99, 3, 5, 5],
                           [120, 3, 3, 2]])
sob = pd.DataFrame(sell_orderbook, columns = ["BP", "Volume", "Latency", "ID"], index = sell_orderbook[:, -1])
bob = sob.copy()

In [153]:
if bo["BP"].values > any(sob.iloc[:, 0]):
    print("price_match")
    temp = sob[sob["BP"].values <= bo["BP"].values]
    temp = temp.sort_values(["BP", "Latency"], ascending = [True, True])
    for index, order in temp.iterrows():
        if bo["Volume"].values > order["Volume"]:
            trade_volume = order["Volume"].copy()
        else:
            trade_volume = bo["Volume"].copy()
        bo["Volume"] -= trade_volume
        sob.at[index, 'Volume'] -= trade_volume
        # agents[sob["ID"]].all_trades.append([order["BP"], - trade_volume])
        
sob = sob[sob["Volume"] > 0]
if bo["Volume"].values > 0:
    bob = bob.append(bo)

price_match


In [147]:
display(bob)
display(sob)

,BP,Volume,Latency,ID
0.0,100.0,5.0,1.0,0.0
1.0,99.0,3.0,2.0,1.0
5.0,99.0,3.0,5.0,5.0
2.0,120.0,3.0,3.0,2.0
3.0,99.5,1.0,4.0,3.0


,BP,Volume,Latency,ID
0,100,5,1,0
2,120,3,3,2
